In [47]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("solutions").getOrCreate()

In [83]:
from pyspark.sql.functions import sum , col ,length

In [49]:
#load datasets 

data_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/revised_final_data1.csv"  
df = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/goods_classification.csv"  
df_goods = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/country_classification.csv"  
df_country = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/services_classification.csv"  
df_services = spark.read.csv(data_path, header=True, inferSchema=True)

In [52]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [53]:
df.withColumn('value', col('value').cast('int'))

DataFrame[time_ref: date, account: string, code: string, country_code: string, product_type: string, value: int, status: string]

In [54]:
df.show()

+----------+-------+----+------------+------------+-------------+------+
|  time_ref|account|code|country_code|product_type|        value|status|
+----------+-------+----+------------+------------+-------------+------+
|2023-06-30|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|2023-06-30|Exports|  00|          AG|       Goods|     351919.0|     F|
|2023-06-30|Exports|  00|          AI|       Goods|      84762.0|     F|
|2023-06-30|Exports|  00|          AL|       Goods|       3463.0|     F|
|2023-06-30|Exports|  00|          AM|       Goods|     679586.0|     F|
|2023-06-30|Exports|  00|          AO|       Goods|     464583.0|     F|
|2023-06-30|Exports|  00|          AR|       Goods|    5943055.0|     F|
|2023-06-30|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|2023-06-30|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|2023-06-30|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|2023-06-30|Exports|  00|          AW|       Goods|

6. Using a UDF, calculate the total export value for each product type in the "revised_csv" file for the June 2023 quarter.

In [55]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [56]:
# Create a Spark session
spark = SparkSession.builder.appName("ExportValueByProductType").getOrCreate()

23/09/06 08:59:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [57]:
# Create a Spark session
spark = SparkSession.builder.appName("ExportValueByProductType").getOrCreate()

In [58]:
# Load the "revised_csv" data
revised_df = spark.read.csv("/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/country_classification.csv", header=True, inferSchema=True)

In [59]:
df.show()

+----------+-------+----+------------+------------+-------------+------+
|  time_ref|account|code|country_code|product_type|        value|status|
+----------+-------+----+------------+------------+-------------+------+
|2023-06-30|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|2023-06-30|Exports|  00|          AG|       Goods|     351919.0|     F|
|2023-06-30|Exports|  00|          AI|       Goods|      84762.0|     F|
|2023-06-30|Exports|  00|          AL|       Goods|       3463.0|     F|
|2023-06-30|Exports|  00|          AM|       Goods|     679586.0|     F|
|2023-06-30|Exports|  00|          AO|       Goods|     464583.0|     F|
|2023-06-30|Exports|  00|          AR|       Goods|    5943055.0|     F|
|2023-06-30|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|2023-06-30|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|2023-06-30|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|2023-06-30|Exports|  00|          AW|       Goods|

In [60]:
# Define a UDF to calculate the total export value, handling non-numeric values
def calculate_total_export_value(values):
    # Initialize a sum variable
    total = 0.0
    
    # Iterate through the values and sum them if they are numeric
    for val in values:
        try:
            total += float(val)
        except (ValueError, TypeError):
            pass  # Skip non-numeric or null values
    
    return total

# Register the UDF
calculate_total_export_udf = udf(calculate_total_export_value, DoubleType())

# Calculate the total export value for each product type in the June 2023 quarter
result_df = df.filter(df.time_ref == "2023-06-30") \
    .groupBy("product_type") \
    .agg(calculate_total_export_udf(F.collect_list("value")).alias("total_export_value"))

# Format the numeric values with 1 decimal place
result_df = result_df.withColumn("total_export_value", F.format_number("total_export_value", 1))

In [61]:
# Show the result
result_df.show()

+------------+------------------+
|product_type|total_export_value|
+------------+------------------+
|    Services|  38,639,599,055.0|
|       Goods| 152,435,617,373.0|
+------------+------------------+



In [62]:
# Stop SparkSession
spark.stop()

7.Generate a pivot table that shows the total import values of Services for the top 4 importing countries in June 2023, with countries as rows and the "Transportation" service categories as columns.

In [72]:
# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("ImportServicePivot").getOrCreate()

23/09/06 09:06:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [73]:
# Load the relevant datasets
revised_csv_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/revised_final_data.csv"
service_csv_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/services_classification.csv"

In [74]:
# Read the revised_csv and service_csv
revised_df = spark.read.csv(revised_csv_path, header=True, inferSchema=True)
service_df = spark.read.csv(service_csv_path, header=True, inferSchema=True)

In [75]:
# Filter the dataset for imports in June 2023-06-30
import_df = revised_df.filter((col("account") == "Imports") & (col("time_ref") == "2023-06-30"))

In [76]:
# Join the import_df with service_df on the "code" column
joined_df = import_df.join(service_df, import_df["code"] == service_df["code"])

In [77]:
# Group by country_code and service_label, and sum the import values
grouped_df = joined_df.groupBy("country_code", "service_label").sum("value")

In [78]:
 #Rank the countries by total import values in descending order
ranked_df = grouped_df.orderBy(col("sum(value)").desc())

In [79]:
# Extract the top 5 importing countries
top_countries = ranked_df.select("country_code").limit(4)

In [80]:
# Pivot the data to create the desired pivot table
pivot_table = ranked_df \
    .join(top_countries, ["country_code"], "inner") \
    .groupBy("country_code").pivot("service_label", ["Services", "Transportation"]) \
    .sum("sum(value)")

In [81]:
# Show the resulting pivot table
pivot_table.show()

+------------+-------------+--------------+
|country_code|     Services|Transportation|
+------------+-------------+--------------+
|          AU|1.881982847E9|  2.49329771E8|
|          US|1.018642986E9|   7.6008244E7|
|          XX| 6.81217569E8|   6.3025441E7|
|          SG| 9.09750687E8|  3.26007453E8|
+------------+-------------+--------------+



In [82]:
# Stop the Spark session
spark.stop()

Q.8 Calculate the total import value of "Services" for each country in June 2023-06-30, and rank the countries in descending order of import value.

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType

# Create a Spark session
spark = SparkSession.builder.appName("ImportServicesValue").getOrCreate()

In [40]:
# Load the relevant datasets
revised_csv_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/revised_final_data1.csv"
service_csv_path = "/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/services_classification.csv"

In [41]:
# Read the revised_csv and service_csv
revised_df = spark.read.csv(revised_csv_path, header=True, inferSchema=True)
service_df = spark.read.csv(service_csv_path, header=True, inferSchema=True)

In [42]:
# Assuming the 'revised_df' DataFrame contains your data
import_df = revised_df


In [43]:
# Filter the DataFrame for imports in June 2023-06-30 and 'Services'
import_df = import_df.filter((col("account") == "Imports") & (col("time_ref") == "2023-06-30") & (col("product_type") == "Services"))


In [44]:
 #Group by 'country_code' and calculate the sum of 'value' for each group
import_df = import_df.groupBy("country_code").agg(sum("value").alias("Total Import Value"))

# Rank the countries in descending order of import value
import_df = import_df.orderBy(col("Total Import Value").desc())

# Rename the columns as per your desired output
import_df = import_df.withColumnRenamed("country_code", "Country Code")

In [45]:
# Show the resulting DataFrame
import_df.show()

+------------+------------------+
|Country Code|Total Import Value|
+------------+------------------+
|          AU|     6.571340755E9|
|          US|      3.46553267E9|
|          XX|      2.03107177E9|
|          SG|     1.831464711E9|
|          CH|     1.136472799E9|
|          GB|     1.116098864E9|
|          DK|     1.080059039E9|
|          CN|      6.56124353E8|
|          IE|      3.62344309E8|
|          IN|      3.32702047E8|
|          CK|      2.83971901E8|
|          JP|      2.45157897E8|
|          BM|      2.13337641E8|
|          PH|      1.95167659E8|
|          WS|      1.93461138E8|
|          FR|      1.78350741E8|
|          ID|      1.72441524E8|
|          HK|      1.69274501E8|
|          DE|      1.52909324E8|
|          CA|      1.51301869E8|
+------------+------------------+
only showing top 20 rows



In [46]:
# Stop the Spark session
spark.stop()

9.Find the top 3 countries with the highest average export value of "Goods" for each month in the updated time_ref (2023-06-30). Use window functions to calculate the average and rank the countries accordingly

In [119]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg, rank

In [120]:
# Initialize SparkSession
spark = SparkSession.builder.appName("Top3CountriesByAvgExport").getOrCreate()

23/09/06 10:27:17 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:75)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:484)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.command

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:556)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.netBind(ServerSocketChannelImpl.java:344)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:301)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerConte

In [86]:
# Read the CSV file into a DataFrame
df = spark.read.option("header", "true").csv("/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/revised_final_data1.csv")

In [87]:
# Filter data for "Goods" category and updated time_ref (2023-06-30)
filtered_df = df.filter((col("product_type") == "Goods") & (col("time_ref") == "2023-06-30"))

In [88]:
# Define a window specification to partition by month and order by average export value
window_spec = Window.partitionBy("time_ref").orderBy(col("avg_export_value").desc())

In [89]:
# Calculate the average export value for each country in each month
avg_export_df = (
    filtered_df
    .groupBy("time_ref", "country_code")
    .agg(avg(col("value")).alias("avg_export_value"))
    .withColumn("rank", rank().over(window_spec))
)

# Filter the top 3 countries for each month
top3_countries_by_month = avg_export_df.filter(col("rank") <= 3)

In [90]:
# Show the result
top3_countries_by_month.show()

+----------+-------------+----------------+----+
|  time_ref| country_code|avg_export_value|rank|
+----------+-------------+----------------+----+
|2023-06-30|TOT (OMT CIF)| 1.9576711757E10|   1|
|2023-06-30|TOT (OMT VFD)| 1.8547416973E10|   2|
|2023-06-30|           DZ|    7.94222382E7|   3|
+----------+-------------+----------------+----+



In [91]:
# Stop SparkSession
spark.stop()

10.Calculate the total import value for each country in the 'revised_final' table. Show the top 10 countries with the highest total export values.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc

# Create a Spark session
spark = SparkSession.builder.appName("TradeAnalysis").getOrCreate()

# Load the 'revised_final' table from your dataset
revised_final = spark.read.csv("path_to_revised_final.csv", header=True, inferSchema=True)

# Filter the data for imports (account = "Imports") and time_ref = "2023-06-30"
import_data = revised_final.filter((revised_final["account"] == "Imports") & (revised_final["time_ref"] == "2023-06-30"))

# Calculate the total import value for each country
total_imports = import_data.groupBy("country_code").agg(sum("value").alias("total_import_value"))

# Show the top 10 countries with the highest total import values
top_10_importing_countries = total_imports.orderBy(desc("total_import_value")).limit(10)

# Show the output
top_10_importing_countries.show()


In [95]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc

# Create a Spark session
spark = SparkSession.builder.appName("TradeAnalysis").getOrCreate()


In [96]:
# Load the 'revised_final' table from your dataset
revised_final = spark.read.csv("/Users/anujkhadka/Fusemachines47/ALL SPARK/Spark_Group_project/data/revised_final_data1.csv", header=True, inferSchema=True)

In [97]:
# Filter the data for imports (account = "Imports") and time_ref = "2023-06-30"
import_data = revised_final.filter((revised_final["account"] == "Imports") & (revised_final["time_ref"] == "2023-06-30"))


In [98]:

# Calculate the total import value for each country
total_imports = import_data.groupBy("country_code").agg(sum("value").alias("total_import_value"))

In [99]:
# Show the top 10 countries with the highest total import values
top_10_importing_countries = total_imports.orderBy(desc("total_import_value")).limit(10)

In [100]:
# Show the output
top_10_importing_countries.show()

+-------------+------------------+
| country_code|total_import_value|
+-------------+------------------+
|TOT (OMT CIF)|   1.9576711757E10|
|TOT (OMT VFD)|   1.8547416973E10|
|           AU|   1.2831898537E10|
|           CN|   1.1493681947E10|
|           US|     9.651088289E9|
|           SG|     4.870192374E9|
|           JP|     4.309482252E9|
|           KR|     3.588781722E9|
|           GB|      2.65543796E9|
|           MY|     2.601556325E9|
+-------------+------------------+



In [101]:
# Stop SparkSession
spark.stop()